In [1]:
import geopandas as gpd
from shapely import wkt
import pandas as pd
import rasterio
import rioxarray
from rasterstats import zonal_stats
import requests
import io

In [2]:
# City boundaries data
osm_cities = gpd.read_file("../data/osm_data_updated.csv") ## not pushed to github due to its size
osm_cities.geometry = osm_cities.geometry.apply(wkt.loads)
osm_cities = gpd.GeoDataFrame(osm_cities,geometry='geometry')
osm_cities = osm_cities.set_crs(epsg=4326)
osm_cities['country'] = [x.split(' ')[0] for x in osm_cities.locode]

## What-a-waste data
waste_cities = pd.read_csv("../data/whatawaste/city_level_data_0_0.csv")
waste_country = pd.read_csv("../data/whatawaste/country_level_data.csv")

## World boundaries
boundaries = gpd.read_file("../data/world-administrative-boundaries.geojson")
osm_cities = osm_cities.merge(boundaries[['iso3','iso_3166_1_alpha_2_codes']].drop_duplicates(),how='left',left_on='country',right_on='iso_3166_1_alpha_2_codes')

In [3]:
import difflib 

def fuzzy_merge(df1, df2, left_on, right_on, how='inner', cutoff=0.9):
    df_other= df2.copy()
    df_other[left_on] = [get_closest_match(x, df1[left_on], cutoff) 
                         for x in df_other[right_on]]
    return df1.merge(df_other, on=left_on, how=how)

def get_closest_match(x, other, cutoff):
    matches = difflib.get_close_matches(x, other, n=1, cutoff=cutoff)
    return matches[0] if matches else None

In [4]:
osm_cities.loc[osm_cities.display_name=='Washington, District of Columbia, United States','name'] = 'Washington Dc' ## Hard-coding sue to there being many washingtons
osm_cities['fuzzy_name'] = [str(x)+' '+str(y) for x,y in zip(osm_cities['name'],osm_cities['iso3'])]
waste_cities['fuzzy_name'] = [str(x)+' '+str(y) for x,y in zip(waste_cities['city_name'],waste_cities['iso3c'])]

In [5]:
waste_cities.drop_duplicates().shape

(367, 114)

In [6]:
basic_merge = waste_cities.merge(osm_cities,how='inner',left_on=['city_name','iso3c'],right_on=['name','iso3'])
print(basic_merge.shape[0])
print(basic_merge.drop_duplicates(subset=['city_name','country_name']).shape[0])

125
124


In [7]:
unmerged_cities = waste_cities[~waste_cities.city_name.isin(basic_merge.city_name)]

In [8]:
unmerged_cities.head()

,iso3c,region_id,country_name,income_id,city_name,additional_data_annual_budget_for_waste_management_year,additional_data_annual_solid_waste_budget_year,additional_data_annual_swm_budget_2017_year,additional_data_annual_swm_budget_year,additional_data_annual_waste_budget_year,...,waste_treatment_controlled_landfill_percent,waste_treatment_incineration_percent,waste_treatment_landfill_unspecified_percent,waste_treatment_open_dump_percent,waste_treatment_other_percent,waste_treatment_recycling_percent,waste_treatment_sanitary_landfill_landfill_gas_system_percent,waste_treatment_unaccounted_for_percent,waste_treatment_waterways_marine_percent,fuzzy_name
1,AFG,SAS,Afghanistan,LIC,Kandahar,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,100.0,NaN,NaN,NaN,NaN,NaN,Kandahar AFG
2,AFG,SAS,Afghanistan,LIC,Mazar-E-Sharif,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,100.0,NaN,NaN,NaN,NaN,NaN,Mazar-E-Sharif AFG
3,AFG,SAS,Afghanistan,LIC,Kabul,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,75.0,NaN,NaN,NaN,25.0,NaN,Kabul AFG
4,AFG,SAS,Afghanistan,LIC,HiratÂ,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,100.0,NaN,NaN,NaN,NaN,NaN,HiratÂ AFG
6,ALB,ECS,Albania,UMC,Korca,NaN,NaN,NaN,NaN,NaN,...,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Korca ALB


In [9]:
merged_cities = fuzzy_merge(unmerged_cities, osm_cities, left_on='fuzzy_name',right_on='fuzzy_name',cutoff=0.91)
merged_cities = pd.concat([basic_merge,merged_cities])

In [10]:
def get_geotiff_uri(country_code, year):
    if country_code == 'XKX':
        country_code = 'KOS' # exception
    
    base_url = 'https://hub.worldpop.org/rest/data/pop/wpic1km'

    if country_code in ['MCO', 'MHL', 'VAT', 'TKL', 'TUV']:
        base_url = 'https://hub.worldpop.org/rest/data/pop/wpgp' # need to increase resolution for tiny countries

    metadata = requests.get(base_url).json()['data']

    for x in metadata:
        if x['iso3'] == country_code and int(x['popyear']) == year:
            file_uris = requests.get(base_url, params={'id': x['id']}).json()['data']['files']

            for file_uri in file_uris:
                if file_uri.lower().endswith('.tif'):
                    return file_uri
    
    raise Exception(f'Worldpop data does not exist for {country_code}')

In [11]:
def get_pop_raster(iso, year):
    uri = get_geotiff_uri(iso,year)
    blob = requests.get(uri).content
    population_raster = rasterio.MemoryFile(io.BytesIO(blob)).open()
    return population_raster

In [12]:
merged_cities = gpd.GeoDataFrame(merged_cities,geometry='geometry')

In [13]:
countries = merged_cities.iso3c.unique()

In [14]:
countries

array(['AFG', 'AGO', 'ALB', 'ARE', 'ARG', 'ARM', 'AUS', 'AZE', 'BEL',
       'BEN', 'BGR', 'BIH', 'BLZ', 'BOL', 'CAN', 'CHE', 'COD', 'COL',
       'CRI', 'CYP', 'CZE', 'DOM', 'DZA', 'EGY', 'ESP', 'EST', 'ETH',
       'FIN', 'GAB', 'GBR', 'GHA', 'GIN', 'GMB', 'GTM', 'HND', 'HUN',
       'IND', 'IRL', 'IRN', 'IRQ', 'ITA', 'JPN', 'KAZ', 'KEN', 'KIR',
       'LBY', 'LKA', 'LTU', 'MAR', 'MEX', 'MKD', 'MLI', 'MMR', 'MNE',
       'MNG', 'MOZ', 'MRT', 'MYS', 'NAM', 'NER', 'NIC', 'NLD', 'NOR',
       'NPL', 'OMN', 'PHL', 'PLW', 'PRT', 'PRY', 'ROU', 'RUS', 'RWA',
       'SEN', 'SLV', 'SRB', 'SVK', 'SVN', 'SWE', 'TJK', 'TUN', 'TUV',
       'UGA', 'URY', 'USA', 'UZB', 'VEN', 'ZWE', 'BRA', 'TZA', 'UKR'],
      dtype=object)

In [15]:
import gc
import time
gc.collect()

0

In [16]:
stats_dfs = []
missing_2016 = []
missing_2020 = []
for c in countries:
    print(c)
    tmp_cities = merged_cities[merged_cities.iso3c == c]
    tmp_cities['pop_2016'] = 0
    tmp_cities['pop_2020'] = 0
    try:
        print(2016)
        pop_2016 = get_pop_raster(c,2016)
        stats_2016 = zonal_stats(tmp_cities,pop_2016.read(1),affine=pop_2016.transform,stats=['sum'])
        tmp_cities['pop_2016'] = [x['sum'] for x in stats_2016]
        del(pop_2016) # to save memory
        time.sleep(5)
    except:
        missing_2016.append(c)
    try:
        print(2020)
        pop_2020 = get_pop_raster(c,2020)
        stats_2020 = zonal_stats(tmp_cities,pop_2020.read(1),affine=pop_2020.transform,stats=['sum'])
        tmp_cities['pop_2020'] = [x['sum'] for x in stats_2020]
        del(pop_2020) # to save memory
        time.sleep(5)
    except:
        missing_2020.append(c)
    stats_dfs.append(tmp_cities)

cities_w_pop_stats = pd.concat(stats_dfs)

AFG
2016


/home/kaushik/.local/share/virtualenvs/oef-hackday-q2-24-OS3fY_pt/lib/python3.9/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/home/kaushik/.local/share/virtualenvs/oef-hackday-q2-24-OS3fY_pt/lib/python3.9/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
ERROR 1: LZWDecode:Wrong length of decoded string: data probably corrupted at scanline 512
ERROR 1: TIFF

2020


ERROR 1: LZWDecode:Wrong length of decoded string: data probably corrupted at scanline 512
ERROR 1: TIFFReadEncodedTile() failed.
ERROR 1: /vsimem/4ac85097-28a5-43e6-b091-f560b0970710/4ac85097-28a5-43e6-b091-f560b0970710.tif, band 1: IReadBlock failed at X offset 1, Y offset 0: TIFFReadEncodedTile() failed.
/home/kaushik/.local/share/virtualenvs/oef-hackday-q2-24-OS3fY_pt/lib/python3.9/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/home/kaushik/.local/share/virtualenvs/oef-hackday-q2-24-OS3fY_pt/lib/python3.9/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

AGO
2016


...

/home/kaushik/.local/share/virtualenvs/oef-hackday-q2-24-OS3fY_pt/lib/python3.9/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


UKR
2016


/home/kaushik/.local/share/virtualenvs/oef-hackday-q2-24-OS3fY_pt/lib/python3.9/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/home/kaushik/.local/share/virtualenvs/oef-hackday-q2-24-OS3fY_pt/lib/python3.9/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/home/kaushik/.local/share/virtualenvs/oef-hackday-q2-24-OS3fY_pt/lib/python3.9/site-packages/geopandas/

2020


/home/kaushik/.local/share/virtualenvs/oef-hackday-q2-24-OS3fY_pt/lib/python3.9/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [24]:
#unsure why some of these don't run consistently. Might be worth saving out all the raster files and just reading them in as needed
print(missing_2016)
print(missing_2020)

['AFG', 'AGO', 'IRN', 'KIR', 'TUN', 'USA', 'ZWE', 'BRA']
['AFG', 'AGO', 'AUS', 'CAN', 'IRN', 'KIR', 'RUS', 'TUN', 'USA', 'ZWE', 'BRA']


In [19]:
cities_w_pop_stats = pd.concat(stats_dfs)

In [20]:
cities_w_pop_stats[cities_w_pop_stats.iso3=="IND"]

,iso3c,region_id,country_name,income_id,city_name,additional_data_annual_budget_for_waste_management_year,additional_data_annual_solid_waste_budget_year,additional_data_annual_swm_budget_2017_year,additional_data_annual_swm_budget_year,additional_data_annual_waste_budget_year,...,bbox_south,bbox_east,bbox_north,country,iso3,iso_3166_1_alpha_2_codes,fuzzy_name_y,fuzzy_name,pop_2016,pop_2020
47,IND,SAS,India,LMC,Pune,NaN,NaN,NaN,NaN,NaN,...,18.429497,74.020214,18.6208699,IN,IND,IN,Pune IND,NaN,4.540728e+06,5.086986e+06
48,IND,SAS,India,LMC,Jodhpur,NaN,NaN,NaN,NaN,NaN,...,26.2059842,73.0888716,26.3536661,IN,IND,IN,Jodhpur IND,NaN,6.437296e+05,7.094598e+05
49,IND,SAS,India,LMC,Nagpur,NaN,NaN,NaN,NaN,NaN,...,21.0529736,79.1788461,21.2310022,IN,IND,IN,Nagpur IND,NaN,2.270217e+06,2.429007e+06
50,IND,SAS,India,LMC,Bengaluru,NaN,NaN,NaN,NaN,NaN,...,12.8340125,77.7840515,13.1436649,IN,IND,IN,Bengaluru IND,NaN,8.579298e+06,9.954064e+06
51,IND,SAS,India,LMC,Kanpur,NaN,NaN,NaN,NaN,NaN,...,26.1761604,80.588372,26.640051,IN,IND,IN,Kanpur IND,NaN,3.662854e+06,3.804405e+06
52,IND,SAS,India,LMC,Nashik,NaN,NaN,NaN,NaN,NaN,...,19.8042133,73.9558464,20.1402331,IN,IND,IN,Nashik IND,NaN,9.974526e+05,1.163316e+06
53,IND,SAS,India,LMC,Dhanbad,NaN,NaN,NaN,NaN,NaN,...,25.1846123,87.757287,25.1925443,IN,IND,IN,Dhanbad IND,NaN,2.657960e+03,3.386041e+03
54,IND,SAS,India,LMC,Ghaziabad,NaN,NaN,NaN,NaN,NaN,...,28.6091315,77.6151037,28.8348273,IN,IND,IN,Ghaziabad IND,NaN,3.227826e+06,3.684897e+06
55,IND,SAS,India,LMC,Navi Mumbai,NaN,NaN,NaN,NaN,NaN,...,18.9984479,73.0497037,19.1894691,IN,IND,IN,Navi Mumbai IND,NaN,6.070490e+05,7.065018e+05


In [21]:
cities_w_pop_stats.to_csv("../data/whatawaste/city_level_data_with_population.csv")